In [ ]:
import pandas as pd
pd.set_option("display.max_rows", 5)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\Price Quote list 20210111.xlsx", None)

In [ ]:
df = df['Components']

In [ ]:
df['Region'] = df['Region'].map(lambda x: 'Russia' if x == 'Russia ' else x)

In [ ]:
df.columns

In [ ]:
df['Customer'] = df['Customer'].map(lambda x: 'MT-System' if x =='Mt System' else x)

In [ ]:
df ['Customer'] = df['Customer'].map(lambda x: x.upper())

In [ ]:
df1 = df[['Date', 'Customer', 'Inpaq PN', 'Inpaq Price(USD/Kpcs)']]

In [ ]:
df1.dropna(subset=['Inpaq Price(USD/Kpcs)'], inplace=True)

In [ ]:
df1

In [ ]:
# 匯入整理好的預算價格，抽出MT-SYSTEM 2021的元件預算來比對價格
budget = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\最終預算報告用表格（新）.xlsx", None)

In [ ]:
budget = budget['Sheet1']

In [ ]:
budget.columns

In [ ]:
budget_sample = budget[(budget['負責業務'] == '許凱智') & (budget['Group'] == 'MT-SYSTEM') & (budget['類型'] == '元件') & (budget['預交年份'] == 2021)]

In [ ]:
budget_sample.columns

In [ ]:
budget_sample.info()

In [ ]:
budget_sample['單價'] = budget_sample['本國幣別NTD'] / budget_sample['數量']

In [ ]:
budget_sample = budget_sample.groupby(['Group', '預交年份', '品名', '單價'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
budget_sample = budget_sample[['Group', '品名', '單價']]

In [ ]:
df1 = df1[['Customer', 'Inpaq PN', 'Inpaq Price(USD/Kpcs)']]
df1 = df1[df1['Customer'] == 'MT-SYSTEM']

In [ ]:
df1.columns = ['Group', '品名', 'Unit price']

In [ ]:
result = pd.merge(budget_sample, df1, on=['Group', '品名'], how='left')

In [ ]:
result[result['Unit price'].notna()]

In [ ]:
'''先寫起來，以後可以拿來當做維護報價表的方式，就不需要去Teams裡面填寫了。
    input按照欄位順序填寫
    然後建立成list再和表格的columns合併成字典
    最後append到表格最後'''
budget_keys = budget.columns
budget_values = ['許凱智', 'TT', 'MT-SYSTEM', 2021, 'March', 20302, 1432423, 'JDS-DS-1234', '元件']
# 下面這步很重要，把輸入的值和欄位名稱合併為dictionary之後才能直接append到dataframe去
combine = dict(zip(budget_keys, budget_values))
# 使用append將上面的dictionary加入表格最後，然後index重新計算
budget_sample.append(combine, ignore_index=True)